# UE boss damage simulation
This attempts to simulate hitting bosses with certain lt configurations.

For now it supports Overkill, Tanya, The collector, Simba and leopard. 

It doesn't consider lt crits since we don't know the proc chance for that and we assume a baseline of crits dealing double damage without any further investment into crit dmg.

In [47]:
BASE_CRIT_DAMAGE = 100 # this means crit deal double dmg by default

In [48]:
import random
import pandas as pd
from typing import List, Tuple, Dict

def simulate_damage(
    base_damage: float = 10.0,
    percent_damage_mod: float = 0.0,
    crit_chance: float = 25.0,
    crit_damage: float = 50.0,
    num_hits: int = 10000,
    use_ok: bool = True,
    ok_proc_chance: float = 10.0,
    collector_multiplier: float = 3.0,
    use_tanya: bool = False,
    tanya_dmg: int = 50,
    use_leopard: bool = False,
    leopard_multiplier: float = 2.0,
    use_simba: bool = False,
    simba_proc_chance: float = 4.0,
) -> Tuple[List[float], List[Dict[str, float]]]:
    """
    Simulate damage for a series of hits, with optional critical hits,
    overkill procs, Tanya procs, and collector procs.
    """
    hits = []
    proc_data = []

    for _ in range(num_hits):
        # Calculate the regular damage with modifiers
        regular_damage = base_damage * (1 + percent_damage_mod / 100)

        is_crit = False
        collector_proc_occurred = False
        tanya_proc_occurred = False
        proc_occurred = False
        proc_depth = 0
        leopard_proc_occured = False
        simba_proc_occured = False

        # Determine if the hit is a critical hit
        if random.random() < crit_chance / 100:
            is_crit = True
            crit_extra_damage = regular_damage * ((crit_damage + BASE_CRIT_DAMAGE) / 100)
            damage = regular_damage + crit_extra_damage
            
            if use_ok:
                # Handle Overkill chain calculation
                multiplier = 1
                while random.random() < ok_proc_chance / 100:
                    proc_occurred = True
                    proc_depth += 1
                    multiplier *= 2
                    if multiplier >= 1000:
                        break
                crit_extra_damage *= multiplier
                damage = regular_damage + crit_extra_damage

            if use_tanya:
                if random.random() < 30 / 100:
                    tanya_proc_occurred = True
                    damage *= 1 + tanya_dmg / 100

            # Handle the collector proc (only on crit)
            if random.random() < 15 / 100:
                collector_proc_occurred = True
                damage *= collector_multiplier
        else:
            damage = regular_damage

        # handle sk procs
        if use_leopard:
            if random.random() < 10 / 100:
                damage *= leopard_multiplier
                leopard_proc_occured = True

        if use_simba:
            if not leopard_proc_occured:
                 if random.random() < simba_proc_chance / 100:
                    damage *= 3
                    simba_proc_occured = True

        hits.append(damage)
        # Build the proc_data dictionary conditionally
        proc_entry = {
            'Damage': damage,
            'IsCrit': is_crit,
            'CollectorProcOccurred': collector_proc_occurred
        }
        
        if use_ok:
            proc_entry['OKProcOccurred'] = proc_occurred
            proc_entry['OKProcDepth'] = proc_depth
        
        if use_tanya:
            proc_entry['TanyaProcOccurred'] = tanya_proc_occurred

        if use_leopard:
            proc_entry['LeopardProcOccurred'] = leopard_proc_occured

        if use_simba:
            proc_entry['SimbaProcOccurred'] = simba_proc_occured

        proc_data.append(proc_entry)

    return hits, proc_data


## MMB simulation, 100% crit chance with Tanya and Collector

In [49]:
# Example usage
num_runs = 5
num_hits = 100000
base_damage = 10
crit_chance = 100
crit_damage = 235
use_ok = False
ok_proc_chance = 24
percent_damage_mod = 75.5
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = True
tanya_dmg = 110

# Initialize lists to store results from each run
all_hits = []
all_proc_data = []

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean      139.933741
std       110.861392
min        76.342500
25%        76.342500
50%        76.342500
75%       160.319250
max       561.117375


Damage      IsCrit  CollectorProcOccurred  TanyaProcOccurred
76.342500   True    False                  False                59426
160.319250  True    False                  True                 25468
267.198750  True    True                   False                10556
561.117375  True    True                   True                  4550
Name: count, dtype: int64


## MMB simulation, 100% crit chance with OK and Collector

In [50]:
# Example usage
num_runs = 5
num_hits = 100000
base_damage = 10
crit_chance = 100
crit_damage = 235
use_ok = True
ok_proc_chance = 24
percent_damage_mod = 75.5
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = False
tanya_dmg = 110

# Initialize lists to store results from each run
all_hits = []
all_proc_data = []

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean      143.483105
std       339.074921
min        76.342500
25%        76.342500
50%        76.342500
75%       135.135000
max     60221.070000


Damage       IsCrit  CollectorProcOccurred  OKProcOccurred  OKProcDepth
76.34250     True    False                  False           0              64669
135.13500    True    False                  True            1              15557
267.19875    True    True                   False           0              11408
252.72000    True    False                  True            2               3704
472.97250    True    True                   True            1               2630
487.89000    True    False                  True            3                847
884.52000    True    True                   True            2                680
958.23000    True    False                  True            4                220
1707.61500   True    True                   True            3                169
1898.910

## MMB simulation, 100% crit chance with Yugo and Collector

In [51]:
# Example usage
num_runs = 5
num_hits = 100000
base_damage = 10
crit_chance = 100
crit_damage = 279
use_ok = False
ok_proc_chance = 24
percent_damage_mod = 122
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = False
tanya_dmg = 110

# Initialize lists to store results from each run
all_hits = []
all_proc_data = []

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean      146.360965
std        95.069226
min       106.338000
25%       106.338000
50%       106.338000
75%       106.338000
max       372.183000


Damage   IsCrit  CollectorProcOccurred
106.338  True    False                    84945
372.183  True    True                     15055
Name: count, dtype: int64
